In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 10000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

im_drone = Image.open('drone4-1.png')
im_drone = im_drone.resize((20,20))

imb = Image.open('bullet.png')
imb = imb.resize((5,5))

for i in range(samples):
    
    switch = random.randint(0,1)
    if switch==0:
        img = Image.new('RGB', (100, 100), color = '#9b7653')
        img.paste(im1,(5,75),im1)
        img.paste(im1,(10,55),im1)
        img.paste(im1,(25,50),im1)
        img.paste(im2,(75,5),im2)
        img.paste(im2,(60,30),im2)
        img.paste(im2,(90,20),im2)
        img.paste(im3,(75,60),im3)
        img.paste(im3,(60,80),im3)
        
        img_b = Image.new('RGB', (100, 100), color = '#9b7653')
        img_b.paste(im1,(5,75),im1)
        img_b.paste(im1,(10,55),im1)
        img_b.paste(im1,(25,50),im1)
        img_b.paste(im2,(75,5),im2)
        img_b.paste(im2,(60,30),im2)
        img_b.paste(im2,(90,20),im2)
        img_b.paste(im3,(75,60),im3)
        img_b.paste(im3,(60,80),im3)
        
        x = random.randint(0,1)
        if x==0:
            img.paste(im_drone,(5,5),im_drone)
            data1.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
            img_b.paste(im_drone,(72,65),im_drone)
            img_b.paste(imb,(80,80),imb)
            data2.append(np.array(np.transpose(np.asarray(img_b),(2,0,1)),dtype=np.float32))
        elif x==1:
            img.paste(im_drone,(40,40),im_drone)
            draw = ImageDraw.Draw(img)
            draw.ellipse((50,50,55,55), fill=(0, 0, 255))
            data1.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
            img_b.paste(im_drone,(72,65),im_drone)
            img_b.paste(imb,(80,80),imb)
            data2.append(np.array(np.transpose(np.asarray(img_b),(2,0,1)),dtype=np.float32))
        #mg.show()
        
    elif switch==1:
        img_blank = Image.new('RGB', (100, 100), color = '#9b7653')
        img_blank.paste(im1,(5,75),im1)
        img_blank.paste(im1,(10,55),im1)
        img_blank.paste(im1,(25,50),im1)
        img_blank.paste(im2,(75,5),im2)
        img_blank.paste(im2,(60,30),im2)
        img_blank.paste(im2,(90,20),im2)
        img_blank.paste(im3,(75,60),im3)
        img_blank.paste(im3,(60,80),im3)
        #mg_blank.show()
        data1.append(np.array(np.transpose(np.asarray(img_blank),(2,0,1)),dtype=np.float32))
        data2.append(np.array(np.transpose(np.asarray(img_blank),(2,0,1)),dtype=np.float32))


#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 5)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


(10000, 3, 100, 100)
epoch [1/200], loss:0.0000
x:  tensor([[1.9917e-08, 6.1810e-06, 6.3367e-05, 1.0611e-08, 9.9987e-01]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.9917e-08, 6.1810e-06, 6.3367e-05, 1.0611e-08, 9.9987e-01]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.9917e-08, 6.1810e-06, 6.3367e-05, 1.0611e-08, 9.9987e-01]],
       grad_fn=<SigmoidBackward>)
epoch [2/200], loss:0.0000
x:  tensor([[3.2261e-08, 1.0000e+00, 1.0000e+00, 8.2990e-08, 7.6494e-12]],
       grad_fn=<SigmoidBackward>)
y: tensor([[3.2261e-08, 1.0000e+00, 1.0000e+00, 8.2990e-08, 7.6494e-12]],
       grad_fn=<SigmoidBackward>)
b: tensor([[3.2261e-08, 1.0000e+00, 1.0000e+00, 8.2990e-08, 7.6494e-12]],
       grad_fn=<SigmoidBackward>)
epoch [3/200], loss:0.0000
x:  tensor([[4.3074e-08, 1.0000e+00, 1.0000e+00, 1.5374e-07, 5.0704e-12]],
       grad_fn=<SigmoidBackward>)
y: tensor([[7.2229e-08, 1.0000e+00, 1.0000e+00, 7.1197e-08, 3.8346e-12]],
       grad_fn=<SigmoidBackward>)
b: tensor([[4.8319e-08, 1.0000e

epoch [25/200], loss:0.0000
x:  tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 2.6694e-04, 8.3708e-11]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 2.6694e-04, 8.3708e-11]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 2.6694e-04, 8.3708e-11]],
       grad_fn=<SigmoidBackward>)
epoch [26/200], loss:-0.0000
x:  tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 4.1190e-19, 2.6255e-10]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 5.9064e-19, 1.9536e-10]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 2.9972e-19, 3.4513e-10]],
       grad_fn=<SigmoidBackward>)
epoch [27/200], loss:-0.0000
x:  tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 3.8752e-19, 4.7864e-10]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 3.8752e-19, 4.7864e-10]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,

epoch [49/200], loss:-0.0000
x:  tensor([[1.0000e+00, 9.9990e-01, 9.9997e-01, 1.0620e-12, 1.1638e-17]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 9.9990e-01, 9.9997e-01, 1.0620e-12, 1.1638e-17]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.0000e+00, 9.9990e-01, 9.9997e-01, 1.0620e-12, 1.1638e-17]],
       grad_fn=<SigmoidBackward>)
epoch [50/200], loss:-0.0000
x:  tensor([[1.0000e+00, 1.0000e+00, 9.9996e-01, 7.2497e-12, 9.0395e-17]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 1.0000e+00, 9.9997e-01, 8.2858e-12, 6.8149e-17]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.0000e+00, 1.0000e+00, 9.9994e-01, 6.5274e-12, 1.4220e-16]],
       grad_fn=<SigmoidBackward>)
epoch [51/200], loss:-0.0000
x:  tensor([[1.0000e+00, 1.0000e+00, 9.9992e-01, 1.0642e-11, 2.2313e-16]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 1.0000e+00, 9.9992e-01, 1.0642e-11, 2.2313e-16]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.0000e+00, 1.0000e+00, 9.9992e-01

epoch [73/200], loss:-0.0000
x:  tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 5.5911e-06, 5.7470e-13]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 5.5911e-06, 5.7470e-13]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 5.5911e-06, 5.7470e-13]],
       grad_fn=<SigmoidBackward>)
epoch [74/200], loss:0.0000
x:  tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 1.5675e-05, 6.1637e-13]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 1.4395e-05, 7.2199e-13]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 1.4431e-05, 1.2658e-12]],
       grad_fn=<SigmoidBackward>)
epoch [75/200], loss:0.0000
x:  tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 3.7851e-05, 2.1013e-12]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 3.7092e-05, 1.7079e-12]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 

epoch [97/200], loss:-0.4321
x:  tensor([[6.7527e-06, 1.0000e+00, 9.9999e-01, 1.0000e+00, 5.7307e-11]],
       grad_fn=<SigmoidBackward>)
y: tensor([[6.7527e-06, 1.0000e+00, 9.9999e-01, 1.0000e+00, 5.7307e-11]],
       grad_fn=<SigmoidBackward>)
b: tensor([[6.7527e-06, 1.0000e+00, 9.9999e-01, 1.0000e+00, 5.7307e-11]],
       grad_fn=<SigmoidBackward>)
epoch [98/200], loss:-0.4321
x:  tensor([[6.4505e-06, 1.0000e+00, 9.9999e-01, 1.0000e+00, 6.1144e-11]],
       grad_fn=<SigmoidBackward>)
y: tensor([[6.4505e-06, 1.0000e+00, 9.9999e-01, 1.0000e+00, 6.1144e-11]],
       grad_fn=<SigmoidBackward>)
b: tensor([[6.4505e-06, 1.0000e+00, 9.9999e-01, 1.0000e+00, 6.1144e-11]],
       grad_fn=<SigmoidBackward>)
epoch [99/200], loss:-0.7440
x:  tensor([[4.4226e-07, 9.9998e-01, 2.8276e-03, 1.0000e+00, 2.1066e-09]],
       grad_fn=<SigmoidBackward>)
y: tensor([[4.4226e-07, 9.9998e-01, 2.8276e-03, 1.0000e+00, 2.1066e-09]],
       grad_fn=<SigmoidBackward>)
b: tensor([[4.4226e-07, 9.9998e-01, 2.8276e-03

epoch [121/200], loss:-0.7500
x:  tensor([[6.2622e-08, 4.4577e-11, 1.0070e-06, 1.0000e+00, 2.2828e-09]],
       grad_fn=<SigmoidBackward>)
y: tensor([[6.2622e-08, 4.4577e-11, 1.0070e-06, 1.0000e+00, 2.2828e-09]],
       grad_fn=<SigmoidBackward>)
b: tensor([[6.2622e-08, 4.4577e-11, 1.0070e-06, 1.0000e+00, 2.2828e-09]],
       grad_fn=<SigmoidBackward>)
epoch [122/200], loss:-0.7370
x:  tensor([[6.6424e-08, 4.6510e-11, 1.0207e-06, 1.0000e+00, 2.4154e-09]],
       grad_fn=<SigmoidBackward>)
y: tensor([[6.6424e-08, 4.6510e-11, 1.0207e-06, 1.0000e+00, 2.4154e-09]],
       grad_fn=<SigmoidBackward>)
b: tensor([[6.6424e-08, 4.6510e-11, 1.0207e-06, 1.0000e+00, 2.4154e-09]],
       grad_fn=<SigmoidBackward>)
epoch [123/200], loss:-0.7427
x:  tensor([[1.0000e+00, 2.3639e-13, 1.0000e+00, 1.0000e+00, 1.9129e-11]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 1.0260e-12, 1.0000e+00, 1.0000e+00, 8.4248e-11]],
       grad_fn=<SigmoidBackward>)
b: tensor([[6.8221e-08, 4.8817e-11, 9.3672e

epoch [145/200], loss:-0.7492
x:  tensor([[2.3088e-07, 1.6084e-09, 6.8333e-07, 9.9999e-01, 1.9628e-07]],
       grad_fn=<SigmoidBackward>)
y: tensor([[2.3088e-07, 1.6084e-09, 6.8333e-07, 9.9999e-01, 1.9628e-07]],
       grad_fn=<SigmoidBackward>)
b: tensor([[2.3088e-07, 1.6084e-09, 6.8333e-07, 9.9999e-01, 1.9628e-07]],
       grad_fn=<SigmoidBackward>)
epoch [146/200], loss:-0.7468
x:  tensor([[2.3663e-07, 2.2562e-09, 7.2428e-07, 9.9999e-01, 2.9654e-07]],
       grad_fn=<SigmoidBackward>)
y: tensor([[2.3663e-07, 2.2562e-09, 7.2428e-07, 9.9999e-01, 2.9654e-07]],
       grad_fn=<SigmoidBackward>)
b: tensor([[2.3663e-07, 2.2562e-09, 7.2428e-07, 9.9999e-01, 2.9654e-07]],
       grad_fn=<SigmoidBackward>)
epoch [147/200], loss:-0.7500
x:  tensor([[1.0000e+00, 3.9449e-10, 1.0000e+00, 1.0000e+00, 7.9832e-08]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 1.2724e-10, 1.0000e+00, 1.0000e+00, 3.0061e-08]],
       grad_fn=<SigmoidBackward>)
b: tensor([[2.5543e-07, 3.2419e-09, 7.0016e

epoch [169/200], loss:-0.9992
x:  tensor([[4.4894e-07, 9.9738e-01, 4.6768e-07, 5.5766e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)
y: tensor([[4.4894e-07, 9.9738e-01, 4.6768e-07, 5.5766e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)
b: tensor([[4.4894e-07, 9.9738e-01, 4.6768e-07, 5.5766e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)
epoch [170/200], loss:-0.9997
x:  tensor([[1.0000e+00, 3.1665e-04, 1.0000e+00, 9.9999e-01, 9.9933e-01]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000, 0.0011, 1.0000, 1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[8.6952e-07, 9.9824e-01, 6.4805e-07, 4.8346e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)
epoch [171/200], loss:-0.9993
x:  tensor([[1.0000, 0.0031, 1.0000, 0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 8.1413e-04, 1.0000e+00, 9.9998e-01, 9.9995e-01]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.3656e-06, 9.9862e-01, 8.3017e-07, 4.3449e-05, 1.0000e+00]],
       grad_fn=<Sigmoid

epoch [193/200], loss:-0.9993
x:  tensor([[1.2906e-06, 9.9993e-01, 2.4763e-06, 2.0560e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.2906e-06, 9.9993e-01, 2.4763e-06, 2.0560e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)
b: tensor([[1.2906e-06, 9.9993e-01, 2.4763e-06, 2.0560e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)
epoch [194/200], loss:-0.9984
x:  tensor([[1.0000e+00, 1.2353e-05, 1.0000e+00, 9.9999e-01, 9.9913e-01]],
       grad_fn=<SigmoidBackward>)
y: tensor([[1.0000e+00, 4.0066e-05, 1.0000e+00, 9.9998e-01, 9.9991e-01]],
       grad_fn=<SigmoidBackward>)
b: tensor([[2.1071e-06, 9.9993e-01, 2.2098e-06, 2.0823e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)
epoch [195/200], loss:-0.9998
x:  tensor([[2.8280e-06, 9.9994e-01, 2.9469e-06, 2.1313e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)
y: tensor([[2.8280e-06, 9.9994e-01, 2.9469e-06, 2.1313e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)
b: tensor([[2.8280e-06, 9.9994e-01, 2.9469e

In [14]:
data3=[]

for i in range(1):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    print("x: ", model(x_1[i].reshape(1,3,100,100)))

x:  tensor([[2.3476e-06, 9.9995e-01, 3.3504e-06, 2.5280e-05, 1.0000e+00]],
       grad_fn=<SigmoidBackward>)


In [13]:
data3=[]

for i in range(1):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(40,40),im_drone)
    draw = ImageDraw.Draw(img)
    draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    print("x: ", model(x_1[i].reshape(1,3,100,100)))

x:  tensor([[1.0000e+00, 1.0574e-05, 1.0000e+00, 9.9999e-01, 9.9873e-01]],
       grad_fn=<SigmoidBackward>)


In [12]:
data3=[]

for i in range(1):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    print("x: ", model(x_1[i].reshape(1,3,100,100)))

x:  tensor([[1.0000e+00, 3.0856e-05, 1.0000e+00, 9.9998e-01, 9.9980e-01]],
       grad_fn=<SigmoidBackward>)


In [11]:
data3=[]

for i in range(1):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    print("x: ", model(x_1[i].reshape(1,3,100,100)))

x:  tensor([[1.0000e+00, 1.1939e-04, 1.0000e+00, 9.9996e-01, 9.9998e-01]],
       grad_fn=<SigmoidBackward>)
